In [2]:
import pandas as pd



# 1. Data loading

In [3]:
SHEET_NAMES = [
    "Millionaire - Easy Q", 
    # "Millionaire - Hard Q"
    ]

In [4]:
datasets = {}
for sheet_name in SHEET_NAMES:
    datasets[sheet_name] = pd.read_excel("data/Trivia-Printable.xlsx", sheet_name=sheet_name)


In [5]:
concatenated_df = pd.DataFrame()
for sheet_name in SHEET_NAMES:
    for suffix in ["", ".1", ".2"]:
        new_df = datasets[sheet_name][["Question" + suffix, "Answer" + suffix]]
        new_df.columns = ["Question", "Answer"]
        new_df = new_df.copy()
        new_df.loc[:, 'Difficulty'] = sheet_name
        concatenated_df = pd.concat([concatenated_df, new_df], axis=0, ignore_index=True)

In [6]:
concatenated_df.to_csv("data/Trivia-Printable_merged.csv", index=False)

# 2. LLM challenger Answer a question

In [6]:
from json import load
import torch
from transformers import pipeline


question = "What is the capital of Sri Lanka?"




MODEL_KWARGS = [
    {"model":"kyutai/helium-1-preview-2b", 
    #  "torch_dtype":torch.float16, 
    #  "device_map":"auto"
     },
    {"model":"mistralai/Mistral-7B-Instruct-v0.3"}, # Memory overload
    {"model":"meta-llama/Llama-3.2-1B", "torch_dtype":torch.float16, "device_map":"auto"}, 
    {"model":"gpt2"},
    {"model":"gpt2-large"},
    {"model":"TinyLlama/TinyLlama-1.1B-Chat-v1.0", "torch_dtype":torch.float16, "device_map":"auto"}, # forever running
    {"model":"HuggingFaceTB/SmolLM2-360M-Instruct"},
    {"model":"google/gemma-2-2b-it"}, # Working!
    # {"model":"mistralai/Mistral-Nemo-Instruct-2407"}
]

PROMPT = """You are participating in a trivia contest. Answer the following question: {question}"""

model_kwargs = MODEL_KWARGS[-1]

# pipe = pipeline(
#     "text-generation", 
#     **model_kwargs
# )
# pipe.save_pretrained("./models/"+model_kwargs["model"])


In [2]:

loaded_pipe = pipeline("text-generation", model="./models/"+model_kwargs["model"])



Loading checkpoint shards: 100%|██████████| 3/3 [00:00<00:00,  7.71it/s]
Device set to use cpu
The 'batch_size' attribute of HybridCache is deprecated and will be removed in v4.49. Use the more precisely named 'self.max_batch_size' attribute instead.


In [8]:
text = loaded_pipe(PROMPT.format(question="What is the capital of Burkina Faso?"), max_length=100)[0]["generated_text"]
text

'You are participating in a trivia contest. Answer the following question: What is the capital of Burkina Faso?\n\n**Answer:** Ouagadougou \n'